# Advent of Code 2024 Solutions

First, let's import all the packages we'll use:

In [1]:
import numpy as np
import re
import networkx as nx
# import copy

# [Day 1 Historian Hysteria](https://adventofcode.com/2024/day/1)

In [7]:
file = open('input1.txt', 'r')
input_text = file.read()
file.close()

## Part 1

Starting off pretty easy - let's just convert our lists to vectors, sort by size, and then take the difference.

In [8]:
def create_lists(input_text):
    lines = input_text.strip().splitlines()
    left_list = []
    right_list = []
    for line in lines:
        left, right = line.split()
        left_list.append(int(left))
        right_list.append(int(right))
    return left_list, right_list

In [11]:
left_list, right_list = create_lists(input_text)
left_list = np.sort(left_list)
right_list = np.sort(right_list)
np.sum(np.absolute(right_list-left_list))

2367773

## Part 2

This one's also pretty chill. We can just loop through each value in left list and count how many times it appears in the right list.

In [13]:
score = 0
for num in left_list:
    score += num * np.count_nonzero(right_list == num)
score

21271939

# [Day 2: Red-Nosed Reports](https://adventofcode.com/2024/day/2)

In [1]:
file = open('input2.txt', 'r')
input_text = file.read()
file.close()

## Part 1

In [2]:
def read_reports(input_text):
    lines = input_text.strip().splitlines()
    
    reports = []
    for line in lines:
        reports.append([int(level) for level in line.split()])
    return reports

In [3]:
reports = read_reports(input_text)

A report is safe if its **strictly monotonic**, and each consecutive difference is **between $1$ and $3$ in absolute value**. We'll create a function to test for this and then run this function on every report in our list.

In [4]:
def is_report_safe(report):
    differences = []
    for i in range(len(report) - 1):
        differences.append(report[i+1]-report[i])
    strictly_monotonic = all(num < 0 for num in differences) or all(num > 0 for num in differences)
    gradual = all(1 <= num and 3 >= num for num in [abs(difference) for difference in differences])
    return strictly_monotonic and gradual

In [5]:
number_safe = 0
for report in reports:
    if is_report_safe(report):
        number_safe += 1
number_safe

502

## Part 2 

For each report, we can see if it only has one bad level by removing each level and seeing if the remaining levels constitute a safe report.

In [7]:
number_safe = 0
for report in reports:
    for i in range(len(report)):
        if is_report_safe([level for j, level in enumerate(report) if j != i]):
            number_safe += 1
            break
number_safe

544

# [Day 3: Mull It Over](https://adventofcode.com/2024/day/3)

In [2]:
file = open('input3.txt', 'r')
input_text = file.read()
file.close()

## Part 1

We can use a simple **regular expression** to detect all substrings of the form `mul(X,Y)` where `X` and `Y` are 1-3 digit numbers

In [3]:
mem = input_text

In [4]:
regex = r"mul\((-?\d+),(-?\d+)\)"
def extract_multiplications(mem):
    total = 0
    matches = re.findall(regex, mem)
    for match in matches:
        total += int(match[0])*int(match[1])
    return total

In [6]:
# mult_regex = r"mul\((-?\d+),(-?\d+)\)"
extract_multiplications(mem)

167090022

## Part 2

We just have to preprocess our string a bit for part 2. We can split our string into blocks separated by `don't()`'s. Since the multiplication instructions are **enabled at the beginning**, we can count all the multiplications in the first block. In the other blocks, we only want to count multiplications **after the first `do()`**, since the start of each block follows a `don't()`.

In [7]:
blocks = mem.split("don't()")
total = 0
for block in blocks[1:]:
    do = str(block.split("do()", 1)[1:])
    total += extract_multiplications(do)
total += extract_multiplications(blocks[0])
total

89823704

# Day 4

In [2]:
file = open('input4.txt', 'r')
input_text = file.read()
file.close()

## Part 1

In [10]:
def read_word_search(input_text):
    lines = input_text.strip().splitlines()

    # We'll pad our array so when we use numpy vector as indices, we don't have to worry about going out of bounds.
    word_search = [['.']*(len(lines[0])+2)] 
    
    for line in lines:
        row = ['.']
        for col_index, char in enumerate(line):
            row.append(char) 
        row.append('.')
        word_search.append(row)
    word_search.append(['.']*(len(lines[0])+2))
    return word_search

In [4]:
word_search = read_word_search(input_text)

For each character in our word search, we'll iterate in each direction and check if those characters spell "XMAS."

In [5]:
def detect_xmas(i, j, direction, word_search):
    current = np.array([i,j])
    for i in range(4):
        ith_letter = current + i * direction
        if word_search[ith_letter[0]][ith_letter[1]] != "XMAS"[i]:
            return 0
    return 1

In [6]:
directions = [np.array(direction) for direction in [(-1,0),(0,1),(1,0),(0,-1),(1,1),(1,-1),(-1,-1),(-1,1)]]    
total = 0
for i, row in enumerate(word_search):
    for j, entry in enumerate(row):
        if entry == '.':
            continue
        for direction in directions:
            total += detect_xmas(i, j, direction, word_search)
total

2406

## Part 2

Not much extra stuff to do here. We just have to get a little fancier to get the neighboring letters. For any given `A`, we just have to check the cells neighboring it diagonally. Below, we flatten out our diagonals from the bottom left to the top right. So if we have an instance of
$\begin{bmatrix} x_1 & . & x_2 \\ .  \end{bmatrix}$

In [7]:
valid_x_mas = {"MSAMS","MMASS","SMASM","SSAMM"}
def detect_x_mas(i, j, word_search):
    cross = ""
    parity_checker = (i+j) % 2
    for k in range(i-1, i+2):
        for l in range(j-1, j+2):
            if (k + l) % 2 == parity_checker: 
                cross += word_search[k][l]
    return 1 if cross in valid_x_mas else 0

In [8]:
x_mas_total = 0
for i, row in enumerate(word_search):
    for j, entry in enumerate(row):
        if entry == '.':
            continue
        x_mas_total += detect_x_mas(i, j, word_search)
x_mas_total

1807